* Master DAC - BDLE
* Author: Mohamed-Amine Baazizi
* Affiliation: LIP6 - Faculté des Sciences - Sorbonne Université
* Email: mohamed-amine.baazizi@lip6.fr
* October 2023



# Map-Reduce in Spark

The goal of this lab is to write programs in the Map-Reduce paradigm using the RDD-python API of Spark.
The API for documentation is available here:

- https://spark.apache.org/docs/latest/api/python/reference/pyspark.html


You are expected to use the following higher order functions like `map, flatMap, filter, reduceByKey, groupByKey, join, zipWithIndex` and the standard relational operators `union, intersection, subtract, join` and to favor functional-style programming.

## Préparation

Vérifier que des ressources de calcul sont allouées à votre notebook est connecté (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/DM1

In [ ]:
!pip install -q pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

local = "local[*]"
appName = "Spark-Dataframe-JSON"
localConfig = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")

spark = SparkSession.builder.config(conf = localConfig).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Démarrer la session spark

## Data loading

In [ ]:
#size of displayed elements
n=10

In [ ]:
! wget https://nuage.lip6.fr/s/BMHNNgi9YoKCmCi/download/wordcount.txt -O /tmp/wordcount.txt

--2023-11-16 13:16:29--  https://nuage.lip6.fr/s/BMHNNgi9YoKCmCi/download/wordcount.txt
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 799738 (781K) [text/plain]
Saving to: ‘/tmp/wordcount.txt’

/tmp/wordcount.txt  100%[===================>] 780.99K  1.24MB/s    in 0.6s    

2023-11-16 13:16:31 (1.24 MB/s) - ‘/tmp/wordcount.txt’ saved [799738/799738]



## Simplified TF*IDF

Write instructions to compute the simplified version of TF*IDF for a synthetic corpus of words.
The input data is an RDD of pairs $$(d_i, [w_j])$$
where $$d_i \text{ is a document identifier and } [w_i] \text{ a list of words associated to } d_i \text{ with possible repetition }$$

You are expected to produce an RDD of triples $$(d_l, w_m,  tf \times idf)$$
where $$tf \text{ is the number of occurrence of } w_m \text{ in } d_l$$
and $$ idf \text{ is the inverse of the number of documents where } w_m \text{ appears }  $$

In [ ]:
dataset = ['d1,one fish two fish','d2,red fish blue', 'd3,one red bird']
data = spark.sparkContext.parallelize(dataset)
lines = data.filter(lambda x: x!='')\
.map(lambda x : x.split(",")).map(lambda x:(x[0],x[1].split(" ")))
lines.take(n)

[('d1', ['one', 'fish', 'two', 'fish']),
 ('d2', ['red', 'fish', 'blue']),
 ('d3', ['one', 'red', 'bird'])]

In [ ]:
count = data.count()
count

3

complete the stub of `combine` which is python function that combines each word in `list` with `word` given as argument

In [ ]:
def combine(list, word):
  res = [(word, l) for l in list]
  return res

In [ ]:
list = ['d3', 'one', 'red', 'bird']
word = 'd1'
combine(list,word)

[('d1', 'd3'), ('d1', 'one'), ('d1', 'red'), ('d1', 'bird')]

starting from `lines` produce an RDD of all pairs $$(d_i, w_j)$$

In [ ]:
# flatMap: Return a new RDD by first applying a function to all elements of this RDD, and then flattening the results.
word_doc = lines.flatMap(lambda x: combine(x[1],x[0]))
word_doc.take(n)

[('d1', 'one'),
 ('d1', 'fish'),
 ('d1', 'two'),
 ('d1', 'fish'),
 ('d2', 'red'),
 ('d2', 'fish'),
 ('d2', 'blue'),
 ('d3', 'one'),
 ('d3', 'red'),
 ('d3', 'bird')]

In [ ]:
def add(a,b):
  return a+b

For each pair (document,word), count its frequency

In [ ]:
freq_word = word_doc.map(lambda x: ((x[1],x[0]),1)).reduceByKey(add)
freq_word.take(n)

[(('one', 'd1'), 1),
 (('fish', 'd2'), 1),
 (('one', 'd3'), 1),
 (('red', 'd3'), 1),
 (('fish', 'd1'), 2),
 (('two', 'd1'), 1),
 (('red', 'd2'), 1),
 (('blue', 'd2'), 1),
 (('bird', 'd3'), 1)]

for each word, produce a pair (d,freq)

In [ ]:
freq_word_bis = freq_word.map(lambda x: (x[0][0],(x[0][1],x[1])))
freq_word_bis.take(n)

[('one', ('d1', 1)),
 ('fish', ('d2', 1)),
 ('one', ('d3', 1)),
 ('red', ('d3', 1)),
 ('fish', ('d1', 2)),
 ('two', ('d1', 1)),
 ('red', ('d2', 1)),
 ('blue', ('d2', 1)),
 ('bird', ('d3', 1))]

for each word, compute its document frequency, i.e # of documents in which it appears

In [ ]:
doc_freq = freq_word_bis.map(lambda x : (x[0],1)).reduceByKey(add).map(lambda x : (x[0],count/x[1]))
doc_freq.take(n)

[('fish', 1.5),
 ('two', 3.0),
 ('bird', 3.0),
 ('one', 1.5),
 ('red', 1.5),
 ('blue', 3.0)]

for each word, compute its simplified score

In [ ]:
final = freq_word_bis.join(doc_freq)
final.take(n)

[('two', (('d1', 1), 3.0)),
 ('bird', (('d3', 1), 3.0)),
 ('red', (('d3', 1), 1.5)),
 ('red', (('d2', 1), 1.5)),
 ('blue', (('d2', 1), 3.0)),
 ('fish', (('d2', 1), 1.5)),
 ('fish', (('d1', 2), 1.5)),
 ('one', (('d1', 1), 1.5)),
 ('one', (('d3', 1), 1.5))]

In [ ]:
final = freq_word_bis.join(doc_freq).map(lambda x: (x[0],x[1][0][0], x[1][1]/x[1][0][1]))
final.take(n)

[('two', 'd1', 3.0),
 ('bird', 'd3', 3.0),
 ('red', 'd3', 1.5),
 ('red', 'd2', 1.5),
 ('blue', 'd2', 3.0),
 ('fish', 'd2', 1.5),
 ('fish', 'd1', 0.75),
 ('one', 'd1', 1.5),
 ('one', 'd3', 1.5)]

## Text analytics

The goal is to compute the SPMI (simplified pointwise mutual information) metric for words in a corpus.
Given two words, a and b,

$$SPMI(a, b) =  P(ab) / P(a) * P(b)$$
where $$P(ab) \text{ is the probability of two words coming one after the other, and } P(a), P(b) \text{ the probabilities of the words a and b, respectively} $$

In [ ]:
def removespecialchars (word) :
    charstoremove = ['[',']','*','#','.','_',':','?','!',',',';','“','”','\n']
    for ch in word:
        if ch in charstoremove:
            word = word.replace(ch,'')
    return word

Read file in utf-8 using use_unicode=False

In [ ]:
dataset =''

In [ ]:
data = spark.sparkContext.textFile("/tmp/wordcount.txt", use_unicode="False")
print("before filtering there are %d lines" % data.count())
# data.take(n)
lines = data.filter(lambda x: x!='')\
.map(lambda x:removespecialchars(x))\
.map(lambda x : x.split(" "))
print("after filtering there are %d lines" % lines.count())


before filtering there are 14594 lines
after filtering there are 12153 lines


retrieve the list of words

In [ ]:
words = lines.flatMap(lambda w:w).filter(lambda w:len(w)>0)
words.take(n)

['The',
 'Project',
 'Gutenberg',
 'EBook',
 'of',
 'Pride',
 'and',
 'Prejudice',
 'by',
 'Jane']

compute the frequency of each word

In [ ]:
wordcount = words.map(lambda x: (x,1)).reduceByKey(add)
wordcount.take(n)

[('The', 285),
 ('Project', 83),
 ('EBook', 3),
 ('of', 3693),
 ('Pride', 6),
 ('Jane', 263),
 ('Austen', 4),
 ('is', 859),
 ('use', 23),
 ('anyone', 29)]

count the number of distinct words

In [ ]:
nb_distinct_words = wordcount.count()
nb_distinct_words

7761

compute the proba of each word

In [ ]:
word_proba = wordcount.map(lambda x: (x[0],x[1]/nb_distinct_words))
word_proba.take(n)

[('The', 0.03672207189795129),
 ('Project', 0.010694498131684061),
 ('EBook', 0.00038654812524159255),
 ('of', 0.47584074217240047),
 ('Pride', 0.0007730962504831851),
 ('Jane', 0.033887385646179616),
 ('Austen', 0.0005153975003221235),
 ('is', 0.110681613194176),
 ('use', 0.0029635356268522097),
 ('anyone', 0.003736631877335395)]

Collocation of words: build an RDD of bigrams. A bigram is a pair of words appearing in consecutive order.

In [ ]:
words_index = words.zipWithIndex().map(lambda x: (x[1],x[0]))
words_index.take(n)

[(0, 'The'),
 (1, 'Project'),
 (2, 'Gutenberg'),
 (3, 'EBook'),
 (4, 'of'),
 (5, 'Pride'),
 (6, 'and'),
 (7, 'Prejudice'),
 (8, 'by'),
 (9, 'Jane')]

In [ ]:
words_index_other = words_index.map(lambda x : (x[0]-1,x[1]))
words_index_other.take(n)

[(-1, 'The'),
 (0, 'Project'),
 (1, 'Gutenberg'),
 (2, 'EBook'),
 (3, 'of'),
 (4, 'Pride'),
 (5, 'and'),
 (6, 'Prejudice'),
 (7, 'by'),
 (8, 'Jane')]

In [ ]:
bigram = words_index.join(words_index_other).map(lambda x :x[1])
bigram.take(n)

[('The', 'Project'),
 ('of', 'Pride'),
 ('by', 'Jane'),
 ('eBook', 'is'),
 ('use', 'of'),
 ('at', 'no'),
 ('with', 'almost'),
 ('whatsoever', 'You'),
 ('it', 'give'),
 ('or', 're-use')]

count the fequency of each bigram

In [ ]:
bigramcount = bigram.map(lambda x: (x,1)).reduceByKey(add)
bigramcount.take(n)

[(('use', 'of'), 8),
 (('at', 'no'), 6),
 (('the', 'terms'), 14),
 (('12', '2019'), 1),
 (('PREJUDICE', 'Produced'), 1),
 (('Volunteers', 'and'), 3),
 (('and', 'Prejudice'), 4),
 (('Chapter', '4'), 2),
 (('Chapter', '10'), 2),
 (('Chapter', '12'), 2)]

attach to each bigram the probability of its words, i.e. $$P(a) * P(b)$$

In [ ]:
bigramproba = bigramcount.map(lambda x : x[0]).join(word_proba).map(lambda x: (x[1][0],(x[0],x[1][1]))).join(word_proba).map(lambda x: ((x[1][0][0],x[0]),x[1][0][1]*x[1][1]))
bigramproba.take(n)

[(('of', 'Pride'), 0.0003678706936006188),
 (('Title', 'Pride'), 9.961296875185996e-08),
 (('vain', 'Pride'), 1.7930334375334792e-06),
 (('mine', 'Pride'), 1.4941945312778994e-06),
 (('cover', 'Pride'), 1.9922593750371993e-07),
 (('of', 'them'), 0.026609313503778098),
 (('like', 'them'), 0.0005476056935518915),
 (('seen', 'them'), 0.0005404003554788403),
 (('before', 'them'), 0.0016139957283634696),
 (('make', 'them'), 0.0012104967962726025)]

count the number of distinct bigrams

In [ ]:
bigram_distinct = bigramcount.count()
bigram_distinct

59099

attach to each bigram its probability, i.e. $$P(ab)$$

In [ ]:
bigramjointproba = bigramcount.map(lambda x: (x[0], x[1]/bigram_distinct))
bigramjointproba.take(n)

[(('use', 'of'), 0.00013536608064434254),
 (('at', 'no'), 0.0001015245604832569),
 (('the', 'terms'), 0.00023689064112759946),
 (('12', '2019'), 1.6920760080542818e-05),
 (('PREJUDICE', 'Produced'), 1.6920760080542818e-05),
 (('Volunteers', 'and'), 5.076228024162845e-05),
 (('and', 'Prejudice'), 6.768304032217127e-05),
 (('Chapter', '4'), 3.3841520161085635e-05),
 (('Chapter', '10'), 3.3841520161085635e-05),
 (('Chapter', '12'), 3.3841520161085635e-05)]

compute the SPMI and sort in descending order on the score

In [ ]:
SPMI = bigramjointproba.join(bigramproba).map(lambda x: (x[0], x[1][0]/x[1][1])).sortBy(lambda x: x[1],ascending = False)
SPMI.take(n)


[(('staff', 'Please'), 1019.1901893433051),
 (('he—poor', 'Eliza—to'), 1019.1901893433051),
 (('net', 'purses'), 1019.1901893433051),
 (('Italian', 'songs'), 1019.1901893433051),
 (('barbarously', 'misused'), 1019.1901893433051),
 (('weighty', 'accusation'), 1019.1901893433051),
 (('vicious', 'propensities—the'), 1019.1901893433051),
 (('untamed', 'unabashed'), 1019.1901893433051),
 (('City', 'UT'), 1019.1901893433051),
 (('Kenilworth', 'Birmingham'), 1019.1901893433051)]

## Matrix multiplication

Write a program to perform matrix multiplication using the Spark RDD operators.
The input matrices are represented with their coordinates: each matrix is an RDD of tuples `(i,j,val)` where `i` is the line number, `j` the column number and `val` the value.
The input matrices are provided (`matM` and `matN`).
The output matrix is also represented with its coordinates:

In [ ]:
M = [(1,1,1),(1,2,2),(1,3,3),(2,1,2),(2,2,5),(2,3,7)]
N = [(1,1,2),(1,2,4),(1,3,8),(2,1,1),(2,2,5),(2,3,10),(3,1,3),(3,2,6),(3,3,9)]

matM = spark.sparkContext.parallelize(M)
matN = spark.sparkContext.parallelize(N)

In [ ]:
matMElem = matM.map(lambda x:(x[1],(x[0],x[2]))) #(i,j,v) -> (j,(i,v))
matNElem = matN.map(lambda x:(x[0],(x[1],x[2]))) #(j,k,v') -> (j,(k,v'))
pairwiseProd = matMElem.join(matNElem) #(j,((i,v),(k,v')))
product = pairwiseProd.map(lambda x:((x[1][0][0],x[1][1][0]),x[1][0][1]*x[1][1][1])).reduceByKey(add) #((i,k),v*v')
product.collect()

[((1, 1), 13),
 ((1, 2), 32),
 ((2, 3), 129),
 ((1, 3), 55),
 ((2, 2), 75),
 ((2, 1), 30)]

##END